In [2]:
import pandas as pd
import datetime as dt
import swifter
from collections import namedtuple
import matplotlib.pyplot as plt
import numpy as np

In [1]:
def parse_mi_ho(df):
    
    nr_pci = ''

    def NR_OTA():
        if df["type_id"].iloc[i] == "5G_NR_RRC_OTA_Packet":
            return True
        else:
            return False

    def find_1st_after(target, look_after=1):
        for j in range(i, len(df)):
            t_ = df["time"].iloc[j]
            if (t_ - t).total_seconds() > look_after:
                return None, None
            if df[target].iloc[j] not in [0,'0'] and not np.isnan(df[target].iloc[j]):
                return t_, j

    def find_1st_before(target, look_before=1):
        for j in range(i, -1, -1):
            t_ = df["time"].iloc[j]
            if (t - t_).total_seconds() > look_before:
                return None, None
            if df[target].iloc[j] not in [0,'0'] and not np.isnan(df[target].iloc[j]):
                return t_, j

    HO = namedtuple('HO','start, end, others', defaults=(None,None))

    D = {
        'Conn_Rel':[], 
        'Conn_Req':[], # Setup
        'LTE_HO': [], # LTE -> newLTE
        'MN_HO': [], # LTE + NR -> newLTE + NR
        'MN_HO_to_eNB': [], # LTE + NR -> newLTE
        'SN_setup': [], # LTE -> LTE + NR => NR setup
        'SN_Rel': [], # LTE + NR -> LTE
        'SN_HO': [], # LTE + NR -> LTE + newNR
        # 'HOF': [], # Didn't defined yet.
        'RLF_II': [],
        'RLF_III': [],
        'SCG_RLF': [],
        }

    for i in range(len(df)):
        if NR_OTA():
            continue

        t = df["time"].iloc[i]
        
        others = ''
        if df["rrcConnectionRelease"].iloc[i] == 1:
            D['Conn_Rel'].append(HO(start=t))

        if df["rrcConnectionRequest"].iloc[i] == 1:
            a = find_1st_after('rrcConnectionReconfigurationComplete',look_after=2)[0]
            b = find_1st_after('securityModeComplete',look_after=2)[0]
            end = a if a > b else b
            D['Conn_Req'].append(HO(start=t,end=end))
            nr_pci = ''
        
        if df["lte-rrc.t304"].iloc[i] == 1:
            end, _ = find_1st_after('rrcConnectionReconfigurationComplete')
            serv_cell, target_cell = df["PCI"].iloc[i], df['lte_targetPhysCellId'].iloc[i]
            serv_freq, target_freq = df["Freq"].iloc[i], df['dl-CarrierFreq'].iloc[i]

            if df["SCellToAddMod-r10"].iloc[i] == 1:
                n =len(str(df["SCellIndex-r10.1"].iloc[i]).split('@'))
                others=f'Set up {n} SCell.'
            
            if serv_freq != target_freq:
                a,b = find_1st_before("rrcConnectionReestablishmentRequest", 1)
                others += " Inter freq. HO."
                if a is not None:
                    others += " Near after RLF."
                
            if df["nr-rrc.t304"].iloc[i] == 1 and df["dualConnectivityPHR: setup (1)"].iloc[i] == 1:
                
                if serv_cell == target_cell and serv_freq == target_freq:
                    D['SN_setup'].append(HO(start=t, end=end, others=others))
                    nr_pci = df['nr_physCellId'].iloc[i]
                else:    
                    D['MN_HO'].append(HO(start=t, end=end, others=others))
                    nr_pci = df['nr_physCellId'].iloc[i]
            else:
                
                if serv_cell == target_cell and serv_freq == target_freq:
                    a, b = find_1st_before("scgFailureInformationNR-r15")
                    if a is not None:
                        others += " Caused by scg-failure."
                    D['SN_Rel'].append(HO(start=t, end=end, others=others))
                    nr_pci = ''
                else:
                    a, b = find_1st_before("rrcConnectionSetup",3)
                    if a is not None:
                        others += 'Near After connection setup'
                    if nr_pci == '':
                        D['LTE_HO'].append(HO(start=t, end=end, others=others))
                    else:
                        D['MN_HO_to_eNB'].append(HO(start=t, end=end, others=others))

        if df["nr-rrc.t304"].iloc[i] == 1 and not df["dualConnectivityPHR: setup (1)"].iloc[i] == 1:
            end, _ = find_1st_after('rrcConnectionReconfigurationComplete')
            D['SN_HO'].append(HO(start=t,end=end))
            nr_pci = df['nr_physCellId'].iloc[i]

        if df["rrcConnectionReestablishmentRequest"].iloc[i] == 1:
            end, _ = find_1st_after('rrcConnectionReestablishmentComplete', look_after=1)
            b, _ = find_1st_after('rrcConnectionReestablishmentReject', look_after=1)
            others = df["reestablishmentCause"].iloc[i]
            c, d = find_1st_before('scgFailureInformationNR-r15', 1)
            if c != None:
                others  += ' caused by scgfailure.'
            if end is not None: 
                # Type II
                D['RLF_II'].append(HO(start=t,end=end,others=others))
            else: 
                # Type III
                D['RLF_III'].append(HO(start=t,end=b,others=others)) # End for Type III?
            
        if df["scgFailureInformationNR-r15"].iloc[i] == 1:
            others = df["failureType-r15"].iloc[i]
            D['SCG_RLF'].append(HO(start=t,others=others))
    
    return D

In [1]:
f = '/home/wmnlab/diag_log_sm07_2023-05-07_12-42-00_rrc.csv'
df = pd.read_csv(f)
df["time"] = df["time"].swifter.apply(lambda x: pd.to_datetime(x) + dt.timedelta(hours=8))

A = parse_mi_ho(df)

In [67]:
A

{'Conn_Rel': [HO(start=Timestamp('2023-02-04 17:01:09.463349'), end=None, others=None),
  HO(start=Timestamp('2023-02-04 17:03:52.314022'), end=None, others=None),
  HO(start=Timestamp('2023-02-04 17:04:10.211422'), end=None, others=None),
  HO(start=Timestamp('2023-02-04 17:04:44.570337'), end=None, others=None),
  HO(start=Timestamp('2023-02-04 17:05:11.630925'), end=None, others=None),
  HO(start=Timestamp('2023-02-04 17:05:52.481661'), end=None, others=None)],
 'Conn_Req': [HO(start=Timestamp('2023-02-04 17:01:38.984036'), end=Timestamp('2023-02-04 17:01:39.143860'), others=None),
  HO(start=Timestamp('2023-02-04 17:03:59.699403'), end=Timestamp('2023-02-04 17:03:59.832062'), others=None),
  HO(start=Timestamp('2023-02-04 17:04:32.309664'), end=Timestamp('2023-02-04 17:04:32.443060'), others=None),
  HO(start=Timestamp('2023-02-04 17:04:59.859292'), end=Timestamp('2023-02-04 17:04:59.995137'), others=None),
  HO(start=Timestamp('2023-02-04 17:05:38.996789'), end=Timestamp('2023-02-